In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")
%matplotlib inline

In [2]:
df = pd.read_csv("heart.csv")
df.shape

(303, 14)

In [3]:
label_name = "target"

In [4]:
X_raw = df.drop(label_name, axis=1)
y = df[[label_name]]

X_raw.shape, y.shape

((303, 13), (303, 1))

In [7]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y_ohe = ohe.fit_transform(y)
y_ohe = y_ohe.toarray()
print(ohe.get_feature_names_out())

['target_0' 'target_1']


In [8]:
y_ohe.shape

(303, 2)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X.values, y_ohe, test_size=0.33, random_state=42)

In [10]:
y_train.shape

(203, 2)

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(X_train.shape[1], 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, y_train.shape[1]),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) 
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=13, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=2, bias=True)
  )
)


In [21]:
logits = model(torch.FloatTensor(X_train))
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print(f"Predicted class: {y_pred[:10]}")

Predicted class: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [13]:
nb_epochs = 100
for epoch in range(nb_epochs+1):
    
    hypothesis = model(torch.FloatTensor(X_train))
#     cost = (y_train * -torch.log(hypothesis)).sum(dim=1).mean()
    cost = F.cross_entropy(hypothesis, torch.FloatTensor(y_train))
    optimizer.zero_grad()
    # gradient
    cost.backward()
    # Weight and bias update
    optimizer.step()

    if epoch % 20 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/100 Cost: 0.863189
Epoch   20/100 Cost: 0.781107
Epoch   40/100 Cost: 0.662524
Epoch   60/100 Cost: 0.642996
Epoch   80/100 Cost: 0.636258
Epoch  100/100 Cost: 0.629738


In [14]:
def test(model, X_test):
    with torch.no_grad():
        output = model(torch.FloatTensor(X_test))
        _, predicted = torch.max(output.data, 1)
        return predicted
    
    
y_predict = test(model, X_test)
y_predict = np.array(y_predict)
y_predict

array([0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [15]:
y_predict.shape

(100,)

In [16]:
y_test_lable = np.argmax(y_test, axis=1)
y_test_lable

array([0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [17]:
y_predict.shape, y_test_lable.shape

((100,), (100,))

In [18]:
(y_predict == y_test_lable).mean()

0.7